In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [ ]:
all_tags = pl.DataFrame(stash.find_tags(fragment="id name"))

ai_tags = all_tags.filter(pl.col("name").str.ends_with("_AI"))
ai_tag_ids = ai_tags.select("id").to_series().to_list()
ai_tags

In [ ]:
# Find scenes with AI_Tagged (ID: 8559) which do not have any AI markers meaning the marker tags (not primary tag) contains tags which parent tag is AI (ID: 8552)
ai_tagged_scenes = stash.find_scenes({
    "tags": {
        "value": "8559",
        "modifier": "INCLUDES",
    },
    "markers_filter": {
        "tags": {
            "value": [],
            "modifier": "INCLUDES",
            "excludes": ai_tag_ids
        }
    }
}, { "per_page": 50 }, fragment="id title tags { id name parents { id name } } scene_markers { id primary_tag { id name } tags { id name parents { id name } } }")

In [ ]:
current = 6
selected_scenes = [ai_tagged_scenes[current]]
selected_scenes[0]["title"]
for scene in selected_scenes:
    print(f"{scene["title"]} {scene["id"]}")
    print("Tags:")
    for tag in scene["tags"]:
        print(f"  {tag["name"]} {tag["id"]}")
    print()
    print("Markers:")
    for marker in scene["scene_markers"]:
        print(f"  {marker["primary_tag"]["name"]} {marker["id"]}")
        for tag in marker["tags"]:
            print(f"    {tag["name"]} {tag["id"]}", end="")
        print()
    print()

In [ ]:
scene_id = selected_scenes[0]["id"]
scene = stash.find_scene(scene_id, fragment="id title tags { id name parents { id name } } scene_markers { id primary_tag { id name parents { id name } } tags { id name } }")

# Get all tags from the scene
all_scene_tags = scene['tags']

# Filter out AI tags (keep only non-AI tags)
non_ai_tags = []
for tag in all_scene_tags:
    # Check if this tag has AI (id: '8552') as a parent
    has_ai_parent = False
    for parent in tag.get('parents', []):
        if parent['id'] == '8552':
            has_ai_parent = True
            break
    
    # AI_Tagged tag is a special case, it should be removed
    if tag['id'] == '8559':
        continue

    # If it doesn't have AI as parent, keep it
    if not has_ai_parent:
        non_ai_tags.append(tag)

# Extract just the IDs for the update
updated_tag_ids = [tag['id'] for tag in non_ai_tags]

print(f"Original tags count: {len(all_scene_tags)}")
print(f"Non-AI tags count: {len(non_ai_tags)}")
print()

print("Non-AI Tags to keep:")
for tag in non_ai_tags:
    print(f"  - {tag['id']}: {tag['name']}")
print()

# Update the scene with only non-AI tags
stash.update_scene({
    "id": scene_id,  # Using the correct scene ID
    "tag_ids": updated_tag_ids
})
